In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('../input/first-gop-debate-twitter-sentiment/Sentiment.csv')
data = data[['text','sentiment']]
data.head()

In [ ]:
data = data[data.sentiment != "Neutral"]
data['sentiment']= pd.get_dummies(data['sentiment'], drop_first = True)
data = data.reset_index(drop=True)
data.head()

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
corpus = []

for i in range(0, data.shape[0]):
    tweet = re.sub('[^a-zA-Z]', ' ', data['text'][i])
    tweet = tweet.lower()
    tweet = word_tokenize(tweet)
    # Reduce words to their root form
    tweet = [WordNetLemmatizer().lemmatize(w) for w in tweet if not w in set(stopwords.words('english'))]
    # Lemmatize verbs by specifying pos
    tweet = [WordNetLemmatizer().lemmatize(w, pos='v') for w in tweet if not w in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

print(corpus[0:3])

##applying TF-IDF to check the most relevant words after preprocessing , so we can remove some of these word to improve the model accuracy

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(corpus)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
pd.set_option('display.max_rows', None)
df.head(20)


In [ ]:
for i in range(0, len(corpus)):
    corpus[i] = re.sub('co','', corpus[i])
    corpus[i] = re.sub('rt','', corpus[i])
    corpus[i] = re.sub('http','', corpus[i])
    

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(corpus)
encoded_docs = tokenizer.texts_to_sequences(corpus)
padded_sequence = pad_sequences(encoded_docs,maxlen=200)


In [ ]:
print(tokenizer.word_index['trump'])

In [ ]:
print(corpus[0])
print(encoded_docs[0])

In [ ]:
print(padded_sequence[0])

In [ ]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from keras.initializers import Constant

vocab_size = len(tokenizer.word_index) + 1
embedding_vector_length = 200


model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,
                    input_length=25) )
model.add(SpatialDropout1D(0.2))
model.add(LSTM(3, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
# converting the targets to numpy array to feed it into the model
target = np.asarray(data['sentiment'])

In [ ]:
MODEL = model.fit(padded_sequence,target,validation_split=0.2, epochs=5, batch_size=256)

In [ ]:
test_word ='love trump'
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
print(prediction)

finally this is the best accuracy i could get after hyperparametes tuning + TF-IDF as guidance for removing some words